 ## 조화색설문 결과 색상 고정 회귀분석
 
    최초 작성일 : 2020/03/19
    작성자 : 조예슬 
 
    < 작성 내용 >
    
        - 선형회귀분석을 사용하여 색상 값과 elementary 간의 상관관계 알아보기

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.metrics import roc_curve
import statsmodels.api as sm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

from IPython.display import Image
import statsmodels.api as sm

# 색상 고정

In [49]:
# 데이터 불러오기

total = pd.read_csv("data/element_Fixedhue_harmony.csv")
total.head(20)

,Unnamed: 0,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,0,Female,20,bright,bright,1,3,+S-V,+S+V,+S+V,-S-V,original,original
1,1,Male,20,harmony,harmony,1,5,+S-V,+S+V,-S-V,original,-S+V,+S+V
2,2,Female,20,bright,bright,1,5,+S-V,+S-V,+S+V,-S-V,-S+V,-S+V
3,3,Male,20,bright,harmony,1,3,+S-V,original,+S+V,+S+V,original,+S-V
4,4,Female,20,bright,bright,2,3,+S-V,+S+V,+S-V,+S-V,+S-V,+S+V
5,5,Female,30,harmony,harmony,1,4,-S+V,original,+S-V,original,-S+V,+S-V
6,6,Female,30,harmony,bright,1,3,-S+V,+S+V,-S+V,-S+V,-S-V,-S+V
7,7,Female,20,bright,bright,2,3,+S-V,-S+V,+S-V,-S+V,-S-V,+S+V
8,8,Female,30,harmony,bright,0,5,+S+V,+S-V,-S+V,-S-V,original,-S-V
9,9,Female,20,harmony,harmony,1,1,+S-V,+S+V,+S-V,+S-V,+S-V,+S-V


In [50]:
# 전처리 필요없는 컬럼 

cols_to_keep = ["age_class"]

In [51]:
# columns "sex" 전처리 
# Female, Male 별로 나뉘어짐

dummy_sex = pd.get_dummies(total["sex"], prefix="sex")
dummy_sex

,sex_Female,sex_Male
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0
...,...,...
91,1,0
92,1,0
93,1,0
94,1,0


In [52]:
# columns "definition" 전처리 
# bright, harmony 별로 나뉘어짐

dummy_definition = pd.get_dummies(total["definition"], prefix="definition")
dummy_definition

,definition_bright,definition_harmony
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
91,1,0
92,0,1
93,1,0
94,0,1


In [53]:
# columns "need" 전처리 
# bright, harmony 별로 나뉘어짐

dummy_need = pd.get_dummies(total["need"], prefix="need")
dummy_need

,need_bright,need_harmony
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0
...,...,...
91,1,0
92,0,1
93,1,0
94,0,1


In [54]:
# columns "interest" 전처리 
# 1, 2, 3, 4, 5 별로 나뉘어짐

dummy_interest = pd.get_dummies(total["interest"], prefix="interest")
dummy_interest

,interest_1,interest_2,interest_3,interest_4,interest_5
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
91,0,0,1,0,0
92,0,0,0,0,1
93,0,0,1,0,0
94,0,0,0,1,0


In [55]:
# columns "purpose" 전처리 
# 0, 1, 2, 3별로 나뉘어짐
# purpose : clothes&cosmetic(0), clothes(1), cosmetic(2), etc(3)

dummy_purpose = pd.get_dummies(total["purpose"], prefix="purpose")
dummy_purpose

,purpose_0,purpose_1,purpose_2,purpose_3
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,0,1,0
...,...,...,...,...
91,0,0,1,0
92,0,0,1,0
93,0,1,0,0
94,0,1,0,0


In [56]:
# 전처리 합치기 

df1 = total[cols_to_keep].join(dummy_sex)
df2 = df1.join(dummy_definition)
df3 = df2.join(dummy_need)
df4 = df3.join(dummy_interest)
df5 = df4.join(dummy_purpose)
df5 # elementary 다 합친거

,age_class,sex_Female,sex_Male,definition_bright,definition_harmony,need_bright,need_harmony,interest_1,interest_2,interest_3,interest_4,interest_5,purpose_0,purpose_1,purpose_2,purpose_3
0,20,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0
1,20,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0
2,20,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0
3,20,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
4,20,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,50,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
92,50,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0
93,50,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0
94,50,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0


## comp

### comp = "-S+V" 일 경우

In [57]:
# comp항목에서 "-S+V"를 투표한 사람들을 1, 나머지를  0

for i in range(int(total.shape[0])):
    if total["comp"][i] == "-S+V":
        total["comp"][i] = 1
    else:
        total["comp"][i] = 0
        
        
# elementary + comp(-S+V) 합치기

sV = df5.join(total["comp"])
sV    

C:\Users\acorn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\acorn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_class,sex_Female,sex_Male,definition_bright,definition_harmony,need_bright,need_harmony,interest_1,interest_2,interest_3,interest_4,interest_5,purpose_0,purpose_1,purpose_2,purpose_3,comp
0,20,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0
1,20,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0
2,20,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0
3,20,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0
4,20,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,50,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
92,50,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1
93,50,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0
94,50,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0


In [59]:
# comp을 제외한 컬럼추출

except_sV2 = sV.columns[:16]
except_sV2 

Index(['age_class', 'sex_Female', 'sex_Male', 'definition_bright',
       'definition_harmony', 'need_bright', 'need_harmony', 'interest_1',
       'interest_2', 'interest_3', 'interest_4', 'interest_5', 'purpose_0',
       'purpose_1', 'purpose_2', 'purpose_3'],
      dtype='object')

In [60]:
# 결측치 확인
np.isnan(sV[except_sV2])

# 결측치 처리 : 결측치 0으로 대체 
sV[except_sV2] = sV[except_sV2].fillna(0)
sV["comp"] = sV["comp"].fillna(0)

In [61]:
# 분석한 결과에 대해 통계자료 자세하게 뽑아주는 것 

model  = sm.Logit(sV["comp"], sV[except_sV2]).fit()
model.summary2()

Optimization terminated successfully.
         Current function value: 0.422926
         Iterations 7


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Logit
=======================================================================================
Model:                      Logit                    Pseudo R-squared:         0.124   
Dependent Variable:         comp                     AIC:                      105.2018
Date:                       2020-03-19 14:15         BIC:                      135.9740
No. Observations:           96                       Log-Likelihood:           -40.601 
Df Model:                   11                       LL-Null:                  -46.327 
Df Residuals:               84                       LLR p-value:              0.40612 
Converged:                  1.0000                   Scale:                    1.0000  
No. Iterations:             7.0000                                                     
---------------------------------------------------------------------------------------
                    Coef.     Std.Err.       z    P>|z|       [0.025         0.975]    
---------------------------------------------------------------------------------------
age_class           0.0282         0.0286  0.9881 0.3231         -0.0278         0.0842
sex_Female         -0.4999  20711353.2520 -0.0000 1.0000  -40593506.9449  40593505.9451
sex_Male           -0.9182  17401589.4623 -0.0000 1.0000  -34106489.5381  34106487.7017
definition_bright  -1.2255  42847836.0130 -0.0000 1.0000  -83980216.6265  83980214.1755
definition_harmony -0.1926  45276731.1382 -0.0000 1.0000  -88740762.5612  88740762.1761
need_bright        -0.8504  28478002.4234 -0.0000 1.0000  -55815859.9520  55815858.2511
need_harmony       -0.5676  28478002.4234 -0.0000 1.0000  -55815859.6692  55815858.5339
interest_1          0.0827 117967528.1042  0.0000 1.0000 -231212106.3467 231212106.5121
interest_2         -0.7651 116000914.9845 -0.0000 1.0000 -227357616.3085 227357614.7783
interest_3         -0.3830 111333586.6414 -0.0000 1.0000 -218209820.4698 218209819.7038
interest_4          0.6112 116000914.9845  0.0000 1.0000 -227357614.9321 227357616.1546
interest_5         -0.9639 119901889.4629 -0.0000 1.0000 -235003385.9895 235003384.0616
purpose_0          -0.0121 109588316.0119 -0.0000 1.0000 -214789152.5218 214789152.4976
purpose_1          -0.8627 109588316.0119 -0.0000 1.0000 -214789153.3724 214789151.6470
purpose_2           0.1562 109588316.0119  0.0000 1.0000 -214789152.3535 214789152.6659
purpose_3          -0.6995 109588316.0119 -0.0000 1.0000 -214789153.2092 214789151.8102
=======================================================================================

"""

In [65]:
# 독립, 종속 변수 지정
y = sV.iloc[:, 16]
X = sm.add_constant(sV[except_sV2])

y
X

,const,age_class,sex_Female,sex_Male,definition_bright,definition_harmony,need_bright,need_harmony,interest_1,interest_2,interest_3,interest_4,interest_5,purpose_0,purpose_1,purpose_2,purpose_3
0,1.0,20,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0
1,1.0,20,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0
2,1.0,20,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0
3,1.0,20,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0
4,1.0,20,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.0,50,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
92,1.0,50,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0
93,1.0,50,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0
94,1.0,50,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0


In [66]:
# OLS Regression Results 알아보기
model = sm.OLS(y, X)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   comp   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9942
Date:                Thu, 19 Mar 2020   Prob (F-statistic):              0.459
Time:                        14:19:01   Log-Likelihood:                -40.026
No. Observations:                  96   AIC:                             104.1
Df Residuals:                      84   BIC:                             134.8
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0166      0

## 분석 결과
